# Quantitative Momentum Strategy

"Momentum investing" means investing in the stocks that have increased in price the most.

For this project, we're going to build an investing strategy that selects the 50 stocks with the highest price momentum. From there, we will calculate recommended trades for an equal-weight portfolio of these 50 stocks.


## Library Imports

The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [ ]:
import pandas as pd
import numpy as np
import requests
import math
from scipy import stats
import xlsxwriter

## Importing Our List of Stocks

As before, we'll need to import our list of stocks and our API token before proceeding. Make sure the script is still in your working directory and import it with the following command:

In [ ]:
from tickers import get_tickers
stocks = get_tickers()

## Making Our First API Call

It's now time to make the first version of our momentum screener!

We need to get one-year price returns for each stock in the universe. Here's how.

In [ ]:
from api_key import IEX_CLOUD_API_TOKEN

symbol = 'aapl,zts'

api_url = f'https://api.iex.cloud/v1/data/core/quote,advanced_stats/{symbol}?token={IEX_CLOUD_API_TOKEN}'
response = requests.get(api_url)
response.status_code

200

## Parsing Our API Call

This API call has all the information we need. We can parse it using the same square-bracket notation as in the first project of this course. Here is an example.

In [ ]:
data = response.json()

## Executing A Batch API Call & Building Our DataFrame

Just like in our first project, it's now time to execute a batch API call and add the information we need to our DataFrame.

We'll start by running the following code cell, which contains some code we already built last time that we can re-use for this project. More specifically, it contains a function called `chunks` that we can use to divide our list of securities into groups of 2000.

In [ ]:
# Function sourced from
# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def chunk (lst, n):
    for i in range(0,len(lst),n):
      yield lst[i:i+n]


def get_stocks_data(stock_list):
    symbol_groups= list(chunk(stock_list,2000))
    symbol_strings = []
    for i in range(0,len(symbol_groups)):
      symbol_strings.append(','.join(symbol_groups[i]))

    for symbol_string in symbol_strings:
        api_url = f'https://api.iex.cloud/v1/data/core/quote,advanced_stats/{symbol_string}?token={IEX_CLOUD_API_TOKEN}'
        data = requests.get(api_url).json()

    stats_index = int(len(data)/2)

    all_data = []
    for i in range(0,stats_index):
        dict_data = dict(ticker = data[i]['symbol'],
        price = data[i]['latestPrice'],
        year1_change = data[i+stats_index]['year1ChangePercent'],
        month1_change = data[i+stats_index]['month1ChangePercent'],
        month3_change = data[i+stats_index]['month3ChangePercent'],
        month6_change = data[i+stats_index]['month6ChangePercent'],
        shares_to_buy = 'N/A')
        all_data.append(dict_data)

    df= pd.DataFrame(all_data)
    return df

Now we need to create a blank DataFrame and add our data to the data frame one-by-one.

In [ ]:
momentum = get_stocks_data(stocks['Ticker'])
momentum

,ticker,price,year1_change,month1_change,month3_change,month6_change,shares_to_buy
0,MMM,91.31,-0.084766,-0.122610,-0.141448,0.063231,N/A
1,AOS,86.50,0.326299,0.003946,0.070649,0.241845,N/A
2,ABT,109.11,0.074434,-0.087457,-0.037472,0.212511,N/A
3,ABBV,162.28,0.037970,-0.089316,0.008593,0.117602,N/A
4,ACN,315.40,0.122633,-0.163970,-0.108194,0.052836,N/A
...,...,...,...,...,...,...,...
498,XYL,128.11,0.240718,0.000703,0.149947,0.429568,N/A
499,YUM,137.80,0.042989,-0.004047,0.072482,0.190492,N/A
500,ZBRA,289.68,-0.053859,0.018207,0.163981,0.399961,N/A
501,ZBH,124.48,-0.053408,-0.014819,0.013359,0.198110,N/A


## Calculating the Number of Shares to Buy

Just like in the last project, we now need to calculate the number of shares we need to buy. The one change we're going to make is wrapping this functionality inside a function, since we'll be using it again later in this Jupyter Notebook.

In [ ]:
def portfolio_input():
    global portfolio_size
    portfolio_size = input ('Enter the size of your portfolio: ')

    while True:
      try:
        float(portfolio_size)
        break
      except ValueError:
        print('Please introduce a number \n')
        portfolio_size = input ('Enter the size of your portfolio: ')

## Building a Momentum Strategy

Real-world quantitative investment firms differentiate between "high quality" and "low quality" momentum stocks:

* High-quality momentum stocks show "slow and steady" outperformance over long periods of time
* Low-quality momentum stocks might not show any momentum for a long time, and then surge upwards.

The reason why high-quality momentum stocks are preferred is because low-quality momentum can often be cause by short-term news that is unlikely to be repeated in the future (such as an FDA approval for a biotechnology company).

To identify high-quality momentum, we're going to build a strategy that selects stocks from the highest percentiles of:

* 1-month price returns
* 3-month price returns
* 6-month price returns
* 1-year price returns

Let's start by building our DataFrame. You'll notice that I use the abbreviation `hqm` often. It stands for `high-quality momentum`.

In [ ]:
momentum[['year1_percentile_change','month1_percentile_change',	'month3_percentile_change','month6_percentile_change','hqm_score']] = 'N/A'
order_columns = ['ticker','price', 'shares_to_buy',	'year1_change','year1_percentile_change',
                 'month6_change','month6_percentile_change','month3_change','month3_percentile_change',
                 'month1_change','month1_percentile_change','hqm_score']
hqm = momentum[order_columns]
hqm

,ticker,price,shares_to_buy,year1_change,year1_percentile_change,month6_change,month6_percentile_change,month3_change,month3_percentile_change,month1_change,month1_percentile_change,hqm_score
0,MMM,91.31,N/A,-0.084766,N/A,0.063231,N/A,-0.141448,N/A,-0.122610,N/A,N/A
1,AOS,86.50,N/A,0.326299,N/A,0.241845,N/A,0.070649,N/A,0.003946,N/A,N/A
2,ABT,109.11,N/A,0.074434,N/A,0.212511,N/A,-0.037472,N/A,-0.087457,N/A,N/A
3,ABBV,162.28,N/A,0.037970,N/A,0.117602,N/A,0.008593,N/A,-0.089316,N/A,N/A
4,ACN,315.40,N/A,0.122633,N/A,0.052836,N/A,-0.108194,N/A,-0.163970,N/A,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...
498,XYL,128.11,N/A,0.240718,N/A,0.429568,N/A,0.149947,N/A,0.000703,N/A,N/A
499,YUM,137.80,N/A,0.042989,N/A,0.190492,N/A,0.072482,N/A,-0.004047,N/A,N/A
500,ZBRA,289.68,N/A,-0.053859,N/A,0.399961,N/A,0.163981,N/A,0.018207,N/A,N/A
501,ZBH,124.48,N/A,-0.053408,N/A,0.198110,N/A,0.013359,N/A,-0.014819,N/A,N/A


## Calculating Momentum Percentiles

We now need to calculate momentum percentile scores for every stock in the universe. More specifically, we need to calculate percentile scores for the following metrics for every stock:

* `One-Year Price Return`
* `Six-Month Price Return`
* `Three-Month Price Return`
* `One-Month Price Return`

Here's how we'll do this:

In [ ]:
time_periods = ['year1','month6','month3','month1']


for row in hqm.index:
  for time_period in time_periods:
    change_col = hqm[f'{time_period}_change']
    percentil_col = hqm.loc[row,f'{time_period}_change']
    hqm.loc[row, f'{time_period}_percentile_change'] = stats.percentileofscore(change_col,
                                                                               percentil_col)/100

hqm

,ticker,price,shares_to_buy,year1_change,year1_percentile_change,month6_change,month6_percentile_change,month3_change,month3_percentile_change,month1_change,month1_percentile_change,hqm_score
0,MMM,91.31,N/A,-0.084766,0.194831,0.063231,0.280318,-0.141448,0.05169,-0.122610,0.043738,N/A
1,AOS,86.50,N/A,0.326299,0.741551,0.241845,0.681909,0.070649,0.616302,0.003946,0.727634,N/A
2,ABT,109.11,N/A,0.074434,0.425447,0.212511,0.632207,-0.037472,0.222664,-0.087457,0.093439,N/A
3,ABBV,162.28,N/A,0.037970,0.377734,0.117602,0.403579,0.008593,0.393638,-0.089316,0.091451,N/A
4,ACN,315.40,N/A,0.122633,0.487078,0.052836,0.260437,-0.108194,0.087475,-0.163970,0.019881,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...
498,XYL,128.11,N/A,0.240718,0.652087,0.429568,0.900596,0.149947,0.829026,0.000703,0.707753,N/A
499,YUM,137.80,N/A,0.042989,0.385686,0.190492,0.576541,0.072482,0.622266,-0.004047,0.662028,N/A
500,ZBRA,289.68,N/A,-0.053859,0.238569,0.399961,0.874751,0.163981,0.856859,0.018207,0.795229,N/A
501,ZBH,124.48,N/A,-0.053408,0.240557,0.198110,0.59841,0.013359,0.409543,-0.014819,0.572565,N/A


## Calculating the HQM Score

We'll now calculate our `HQM Score`, which is the high-quality momentum score that we'll use to filter for stocks in this investing strategy.

The `HQM Score` will be the arithmetic mean of the 4 momentum percentile scores that we calculated in the last section.

To calculate arithmetic mean, we will use the `mean` function from Python's built-in `statistics` module.

In [ ]:
from statistics import mean

for row in hqm.index:
  momentum_percentiles = []
  for time_period in time_periods:
    momentum_percentiles.append(hqm.loc[row, f'{time_period}_percentile_change'])
  hqm.loc[row, 'hqm_score'] = mean(momentum_percentiles)

hqm

,ticker,price,shares_to_buy,year1_change,year1_percentile_change,month6_change,month6_percentile_change,month3_change,month3_percentile_change,month1_change,month1_percentile_change,hqm_score
0,MMM,91.31,N/A,-0.084766,0.194831,0.063231,0.280318,-0.141448,0.05169,-0.122610,0.043738,0.142644
1,AOS,86.50,N/A,0.326299,0.741551,0.241845,0.681909,0.070649,0.616302,0.003946,0.727634,0.691849
2,ABT,109.11,N/A,0.074434,0.425447,0.212511,0.632207,-0.037472,0.222664,-0.087457,0.093439,0.343439
3,ABBV,162.28,N/A,0.037970,0.377734,0.117602,0.403579,0.008593,0.393638,-0.089316,0.091451,0.3166
4,ACN,315.40,N/A,0.122633,0.487078,0.052836,0.260437,-0.108194,0.087475,-0.163970,0.019881,0.213718
...,...,...,...,...,...,...,...,...,...,...,...,...
498,XYL,128.11,N/A,0.240718,0.652087,0.429568,0.900596,0.149947,0.829026,0.000703,0.707753,0.772366
499,YUM,137.80,N/A,0.042989,0.385686,0.190492,0.576541,0.072482,0.622266,-0.004047,0.662028,0.56163
500,ZBRA,289.68,N/A,-0.053859,0.238569,0.399961,0.874751,0.163981,0.856859,0.018207,0.795229,0.691352
501,ZBH,124.48,N/A,-0.053408,0.240557,0.198110,0.59841,0.013359,0.409543,-0.014819,0.572565,0.455268


## Selecting the 50 Best Momentum Stocks

As before, we can identify the 50 best momentum stocks in our universe by sorting the DataFrame on the `HQM Score` column and dropping all but the top 50 entries.

In [ ]:
hqm = hqm.sort_values(by = 'hqm_score', ascending = False)
hqm = hqm[:50].reset_index(drop = True)
hqm

,ticker,price,shares_to_buy,year1_change,year1_percentile_change,month6_change,month6_percentile_change,month3_change,month3_percentile_change,month1_change,month1_percentile_change,hqm_score
0,NRG,73.930,N/A,1.137960,0.988072,0.832809,0.996024,0.468686,0.99006,0.184017,0.998012,0.993042
1,CEG,191.670,N/A,1.528441,0.996024,0.700373,0.982107,0.704214,0.998012,0.141844,0.988072,0.991054
2,GE,154.630,N/A,1.067636,0.986083,0.779977,0.99006,0.499127,0.994036,0.143464,0.99006,0.99006
3,MU,122.520,N/A,0.965752,0.976143,0.776721,0.988072,0.488510,0.992048,0.302445,1.0,0.989066
4,WDC,72.055,N/A,0.953769,0.972167,0.576696,0.970179,0.440956,0.988072,0.153249,0.994036,0.981113
5,CMG,2957.600,N/A,0.721647,0.946322,0.660193,0.980119,0.301818,0.972167,0.086279,0.94831,0.96173
6,ETN,318.500,N/A,1.014450,0.978131,0.537745,0.962227,0.319637,0.974155,0.060818,0.926441,0.960239
7,AVGO,1344.070,N/A,1.226998,0.99006,0.548986,0.964215,0.223768,0.93837,0.077649,0.942346,0.958748
8,META,511.900,N/A,1.328056,0.992048,0.630135,0.976143,0.369829,0.982107,0.032952,0.858847,0.952286
9,AMAT,207.860,N/A,0.858762,0.968191,0.484075,0.944334,0.378943,0.986083,0.036398,0.880716,0.944831


## Calculating the Number of Shares to Buy

We'll use the `portfolio_input` function that we created earlier to accept our portfolio size. Then we will use similar logic in a `for` loop to calculate the number of shares to buy for each stock in our investment universe.

In [ ]:
portfolio_input()

Enter the size of your portfolio: 1000000


In [ ]:
position_size = float(portfolio_size)/len(hqm)

for i in hqm.index:
  hqm.loc[i,'shares_to_buy'] = math.floor(position_size/hqm.loc[i, 'price'])

hqm

,ticker,price,shares_to_buy,year1_change,year1_percentile_change,month6_change,month6_percentile_change,month3_change,month3_percentile_change,month1_change,month1_percentile_change,hqm_score
0,NRG,73.930,270,1.137960,0.988072,0.832809,0.996024,0.468686,0.99006,0.184017,0.998012,0.993042
1,CEG,191.670,104,1.528441,0.996024,0.700373,0.982107,0.704214,0.998012,0.141844,0.988072,0.991054
2,GE,154.630,129,1.067636,0.986083,0.779977,0.99006,0.499127,0.994036,0.143464,0.99006,0.99006
3,MU,122.520,163,0.965752,0.976143,0.776721,0.988072,0.488510,0.992048,0.302445,1.0,0.989066
4,WDC,72.055,277,0.953769,0.972167,0.576696,0.970179,0.440956,0.988072,0.153249,0.994036,0.981113
5,CMG,2957.600,6,0.721647,0.946322,0.660193,0.980119,0.301818,0.972167,0.086279,0.94831,0.96173
6,ETN,318.500,62,1.014450,0.978131,0.537745,0.962227,0.319637,0.974155,0.060818,0.926441,0.960239
7,AVGO,1344.070,14,1.226998,0.99006,0.548986,0.964215,0.223768,0.93837,0.077649,0.942346,0.958748
8,META,511.900,39,1.328056,0.992048,0.630135,0.976143,0.369829,0.982107,0.032952,0.858847,0.952286
9,AMAT,207.860,96,0.858762,0.968191,0.484075,0.944334,0.378943,0.986083,0.036398,0.880716,0.944831


## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

In [ ]:
writer = pd.ExcelWriter('Momentum Strategy.xlsx', engine='xlsxwriter')
hqm.to_excel(writer, sheet_name='Momentum Strategy', index = False)

## Creating the Formats We'll Need For Our .xlsx File

You'll recall from our first project that formats include colors, fonts, and also symbols like % and $. We'll need four main formats for our Excel document:

* String format for tickers
* \$XX.XX format for stock prices
* \$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase

Since we already built our formats in the last section of this course, I've included them below for you. Run this code cell before proceeding.

In [ ]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

In [ ]:
column_formats = {'A': ['Ticker',string_template],
 'B': ['Price',dollar_template ],
 'C': ['Shares to buy', integer_template],
 'D': ['1 Yr Change', percent_template],
 'E': ['1 Yr Percentile Change',percent_template],
 'F': ['6 Month Change',percent_template],
 'G': ['6 Month Percentile Change',percent_template],
 'H': ['3 Month Change',percent_template],
 'I': ['3 Month Percentile Change',percent_template],
 'J': ['1 Month Change',percent_template],
 'K': ['1 Month Percentile Change',percent_template],
 'L': ['HQM Score',percent_template]}

for column in column_formats.keys():
  writer.sheets['Momentum Strategy'].set_column(f'{column}:{column}', 25, column_formats[column][1])
  writer.sheets['Momentum Strategy'].write(f'{column}1',column_formats[column][0], string_template)

## Saving Our Excel Output

As before, saving our Excel output is very easy:

In [ ]:
writer.close()